Idea code name: `S1P10`

In [1]:
from keras.layers import Input, Dense, LSTM, PReLU, Reshape, Dropout, Activation, BatchNormalization, Lambda
from keras.models import model_from_json, Model
import keras.backend as K
from keras import regularizers
import keras

from s1p10_model.resnet8_body import resnet8_body

from s1p10_training_utils import gen_classifier_dataset

import json
import time

import numpy as np
import sklearn

Using TensorFlow backend.


# Global Config

In [2]:
IMAGE_SHAPE = (200, 200, 1)
NUM_LABELS = 10
BINS_EDGE = np.load("./s1p10_data/s1p10_bins_edge.npy")
NUM_CLASSES = len(BINS_EDGE) - 1  

with open('./s1p10_data/s1p10_classes_weight.json', 'r') as fp:
    CLASSES_WEIGHT = json.load(fp)

# 1. Model Acrchitect

1 Image -> ResNet-8 -> Dense regressor -> 5 steering angles

## 1.2 Create Encoder
Encoder is body of ResNet-8 from `Drone-Net` since it's already trained to recognize road curve -> helpful spatial information


In [3]:
encoder = resnet8_body(IMAGE_SHAPE)
# load resnet8 weights
# encoder.load_weights("./s1p10_model/named_resnet8_best_weights.h5", by_name=True)
# for l in encoder.layers:
#     l.trainable = False
# encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## 1.3 Create Decoder
Decoder is a seri of Dense layers with a non-activate Dense layer at the end to perform regression.

## 1.4 Create Model

In [4]:
reshapor = Reshape((1, -1))

def s1p10_model(input_shape):
    """
    Define see-1-predict-10 model
    
    Input:
        input_shape (tuple): shape of input image inputted to encoder
        
    Output:
        keras.Model
    """
    X_in = Input(shape=input_shape, name="image_in")
    
    # extract feature vector
    X_feature = encoder(X_in)
    X_feature = Dropout(0.5)(X_feature)
    
    # apply classifier body
    X_body = Dense(800, activation='relu')(X_feature)
    
    # apply classifier head
    y = []
    for i in range(NUM_LABELS):
        X_body_1 = Dropout(0.5)(X_body)
        out = Dense(NUM_CLASSES, activation='softmax', name="head_%d" % i)(X_body_1)
        y.append(out)
    
    model = Model(inputs=[X_in], outputs=y)
    return model
        

In [5]:
model = s1p10_model(IMAGE_SHAPE)

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_in (InputLayer)           (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
resnet8 (Model)                 (None, 6272)         309088      image_in[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 6272)         0           resnet8[1][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 800)          5018400     dropout_1[0][0]             

In [6]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=['acc'])

# Training

In [7]:
gen_param = {'num_classes': NUM_CLASSES, 
             'num_labels': NUM_LABELS, 
             'bins_edge': BINS_EDGE, 
             'image_shape': IMAGE_SHAPE, 
             'num_samples': None, 
             'data_root_dir': "/home/user/Bureau/Dataset/udacity/", 
             'flip_prob': 0.5}

X_train, y_train = gen_classifier_dataset("./s1p10_data/ext_s1p10_CH2_002_output_training.csv", **gen_param)
# X_train = np.load('./s1p10_data/bottom_half_CH2_training_X.npy')
# y_train_arr = np.load('./s1p10_data/bottom_half_CH2_training_y.npy')  # shape (5, len_dataset, num_classes)
# y_train = [y_train_arr[i, :, :]for i in range(NUM_LABELS)]

X_val, y_val = gen_classifier_dataset("./s1p10_data/ext_s1p10_CH2_002_output_validation.csv", **gen_param)
# X_val = np.load('./s1p10_data/bottom_half_CH2_validation_X.npy')
# y_val_arr = np.load('./s1p10_data/bottom_half_CH2_validation_y.npy')
# y_val = [y_val_arr[i, :, :]for i in range(NUM_LABELS)]

In [8]:
batch_size = 200

time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './s1p10_logs/' + time_str
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')
early_stop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              patience=3,
                                              restore_best_weights=True)

# eva_metric = EVAMetrics()

history = model.fit(x=X_train,
                  y=y_train,
                  epochs=50,
                  validation_data=(X_val, y_val),
                  class_weight=CLASSES_WEIGHT,
                  initial_epoch=0,
                  shuffle=True,
                  batch_size=batch_size,
                  callbacks=[tb_callback, early_stop_cb])

# eva_data = eva_metric.get_data()

Instructions for updating:
Use tf.cast instead.
Train on 18183 samples, validate on 2021 samples
Epoch 1/50
18183/18183 [==============================] - 46s 3ms/step - loss: 38.6368 - head_0_loss: 3.8278 - head_1_loss: 3.8444 - head_2_loss: 3.8809 - head_3_loss: 3.8412 - head_4_loss: 3.8110 - head_5_loss: 3.8678 - head_6_loss: 3.8858 - head_7_loss: 3.8744 - head_8_loss: 3.9132 - head_9_loss: 3.8903 - head_0_acc: 0.1983 - head_1_acc: 0.1944 - head_2_acc: 0.1982 - head_3_acc: 0.2059 - head_4_acc: 0.2074 - head_5_acc: 0.2006 - head_6_acc: 0.2043 - head_7_acc: 0.2016 - head_8_acc: 0.1999 - head_9_acc: 0.1977 - val_loss: 24.6432 - val_head_0_loss: 2.4551 - val_head_1_loss: 2.4405 - val_head_2_loss: 2.4241 - val_head_3_loss: 2.4122 - val_head_4_loss: 2.4287 - val_head_5_loss: 2.4364 - val_head_6_loss: 2.4837 - val_head_7_loss: 2.4809 - val_head_8_loss: 2.5261 - val_head_9_loss: 2.5555 - val_head_0_acc: 0.2865 - val_head_1_acc: 0.2914 - val_head_2_acc: 0.2746 - val_head_3_acc: 0.2959 - val_

Epoch 9/50
18183/18183 [==============================] - 42s 2ms/step - loss: 11.9963 - head_0_loss: 1.3575 - head_1_loss: 1.2341 - head_2_loss: 1.1446 - head_3_loss: 1.0948 - head_4_loss: 1.0921 - head_5_loss: 1.1147 - head_6_loss: 1.1389 - head_7_loss: 1.1879 - head_8_loss: 1.2692 - head_9_loss: 1.3625 - head_0_acc: 0.5392 - head_1_acc: 0.5763 - head_2_acc: 0.5949 - head_3_acc: 0.6095 - head_4_acc: 0.6148 - head_5_acc: 0.6108 - head_6_acc: 0.6017 - head_7_acc: 0.5943 - head_8_acc: 0.5685 - head_9_acc: 0.5507 - val_loss: 12.3057 - val_head_0_loss: 1.3984 - val_head_1_loss: 1.2781 - val_head_2_loss: 1.1681 - val_head_3_loss: 1.1136 - val_head_4_loss: 1.1071 - val_head_5_loss: 1.1223 - val_head_6_loss: 1.1769 - val_head_7_loss: 1.2391 - val_head_8_loss: 1.3036 - val_head_9_loss: 1.3985 - val_head_0_acc: 0.5398 - val_head_1_acc: 0.5720 - val_head_2_acc: 0.6002 - val_head_3_acc: 0.6343 - val_head_4_acc: 0.6299 - val_head_5_acc: 0.6121 - val_head_6_acc: 0.6042 - val_head_7_acc: 0.5898 - v

Epoch 17/50
18183/18183 [==============================] - 42s 2ms/step - loss: 7.6597 - head_0_loss: 0.8977 - head_1_loss: 0.7732 - head_2_loss: 0.7280 - head_3_loss: 0.7053 - head_4_loss: 0.7015 - head_5_loss: 0.7227 - head_6_loss: 0.7271 - head_7_loss: 0.7424 - head_8_loss: 0.7849 - head_9_loss: 0.8770 - head_0_acc: 0.6880 - head_1_acc: 0.7275 - head_2_acc: 0.7352 - head_3_acc: 0.7404 - head_4_acc: 0.7491 - head_5_acc: 0.7383 - head_6_acc: 0.7380 - head_7_acc: 0.7329 - head_8_acc: 0.7269 - head_9_acc: 0.7005 - val_loss: 9.6297 - val_head_0_loss: 1.1116 - val_head_1_loss: 0.9910 - val_head_2_loss: 0.9036 - val_head_3_loss: 0.9009 - val_head_4_loss: 0.8873 - val_head_5_loss: 0.8787 - val_head_6_loss: 0.9101 - val_head_7_loss: 0.9641 - val_head_8_loss: 1.0031 - val_head_9_loss: 1.0793 - val_head_0_acc: 0.6220 - val_head_1_acc: 0.6744 - val_head_2_acc: 0.6883 - val_head_3_acc: 0.6729 - val_head_4_acc: 0.6952 - val_head_5_acc: 0.7051 - val_head_6_acc: 0.6957 - val_head_7_acc: 0.6680 - va

Epoch 25/50
18183/18183 [==============================] - 42s 2ms/step - loss: 5.7341 - head_0_loss: 0.6731 - head_1_loss: 0.5696 - head_2_loss: 0.5452 - head_3_loss: 0.5371 - head_4_loss: 0.5406 - head_5_loss: 0.5411 - head_6_loss: 0.5379 - head_7_loss: 0.5500 - head_8_loss: 0.5836 - head_9_loss: 0.6559 - head_0_acc: 0.7631 - head_1_acc: 0.7962 - head_2_acc: 0.7999 - head_3_acc: 0.7982 - head_4_acc: 0.8031 - head_5_acc: 0.8034 - head_6_acc: 0.8043 - head_7_acc: 0.8010 - head_8_acc: 0.7952 - head_9_acc: 0.7688 - val_loss: 8.9178 - val_head_0_loss: 1.0017 - val_head_1_loss: 0.9016 - val_head_2_loss: 0.8268 - val_head_3_loss: 0.8349 - val_head_4_loss: 0.8414 - val_head_5_loss: 0.8437 - val_head_6_loss: 0.8496 - val_head_7_loss: 0.8995 - val_head_8_loss: 0.9153 - val_head_9_loss: 1.0032 - val_head_0_acc: 0.6739 - val_head_1_acc: 0.7115 - val_head_2_acc: 0.7150 - val_head_3_acc: 0.7199 - val_head_4_acc: 0.7229 - val_head_5_acc: 0.7140 - val_head_6_acc: 0.7160 - val_head_7_acc: 0.7011 - va

Epoch 33/50
18183/18183 [==============================] - 42s 2ms/step - loss: 4.7025 - head_0_loss: 0.5519 - head_1_loss: 0.4670 - head_2_loss: 0.4514 - head_3_loss: 0.4404 - head_4_loss: 0.4384 - head_5_loss: 0.4414 - head_6_loss: 0.4408 - head_7_loss: 0.4572 - head_8_loss: 0.4730 - head_9_loss: 0.5410 - head_0_acc: 0.8004 - head_1_acc: 0.8278 - head_2_acc: 0.8337 - head_3_acc: 0.8373 - head_4_acc: 0.8389 - head_5_acc: 0.8361 - head_6_acc: 0.8384 - head_7_acc: 0.8338 - head_8_acc: 0.8287 - head_9_acc: 0.8080 - val_loss: 8.1964 - val_head_0_loss: 0.9233 - val_head_1_loss: 0.8193 - val_head_2_loss: 0.7571 - val_head_3_loss: 0.7536 - val_head_4_loss: 0.7818 - val_head_5_loss: 0.7700 - val_head_6_loss: 0.7962 - val_head_7_loss: 0.8264 - val_head_8_loss: 0.8454 - val_head_9_loss: 0.9233 - val_head_0_acc: 0.7066 - val_head_1_acc: 0.7392 - val_head_2_acc: 0.7298 - val_head_3_acc: 0.7432 - val_head_4_acc: 0.7318 - val_head_5_acc: 0.7353 - val_head_6_acc: 0.7308 - val_head_7_acc: 0.7254 - va

Epoch 41/50
18183/18183 [==============================] - 42s 2ms/step - loss: 4.0864 - head_0_loss: 0.4775 - head_1_loss: 0.4257 - head_2_loss: 0.3769 - head_3_loss: 0.3871 - head_4_loss: 0.3839 - head_5_loss: 0.3903 - head_6_loss: 0.3782 - head_7_loss: 0.3862 - head_8_loss: 0.4053 - head_9_loss: 0.4752 - head_0_acc: 0.8302 - head_1_acc: 0.8410 - head_2_acc: 0.8580 - head_3_acc: 0.8563 - head_4_acc: 0.8564 - head_5_acc: 0.8565 - head_6_acc: 0.8612 - head_7_acc: 0.8587 - head_8_acc: 0.8543 - head_9_acc: 0.8313 - val_loss: 8.1127 - val_head_0_loss: 0.9182 - val_head_1_loss: 0.8239 - val_head_2_loss: 0.7544 - val_head_3_loss: 0.7539 - val_head_4_loss: 0.7757 - val_head_5_loss: 0.7599 - val_head_6_loss: 0.7782 - val_head_7_loss: 0.8306 - val_head_8_loss: 0.8177 - val_head_9_loss: 0.9002 - val_head_0_acc: 0.7120 - val_head_1_acc: 0.7427 - val_head_2_acc: 0.7382 - val_head_3_acc: 0.7437 - val_head_4_acc: 0.7462 - val_head_5_acc: 0.7481 - val_head_6_acc: 0.7378 - val_head_7_acc: 0.7338 - va

# Save weights

In [9]:
# serialize model to JSON
model_json = model.to_json()
with open(log_dir + "/ext_bottom_half_s1p10_model_%s.json" % time_str, "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(log_dir + "/ext_bottom_half_s1p10_model_%s.h5" % time_str)
print("Saved model to disk")

Saved model to disk


In [10]:
time_str

'2019_06_07_14_01'

In [11]:
#TODO: save X_train, y_train
np.save('./s1p10_data/corrected_ext_CH2_training_X_%s.npy' % time_str, X_train)
np.save('./s1p10_data/corrected_ext_CH2_training_y_%s.npy' % time_str, y_train)

np.save('./s1p10_data/corrected_ext_CH2_validation_X_%s.npy' % time_str, X_val)
np.save('./s1p10_data/corrected_ext_CH2_validation_y_%s.npy' % time_str, y_val)